In [265]:
# Set up packages
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
import random
import numpy as np
import os
from functools import reduce

In [266]:
# open the initial database
wd = "/Users/goldriev/Google Drive/My Drive/Developmental_competence_ms/TriTyrp_ML/"
df = pd.read_csv(wd + 'gene_data.csv', sep=",", na_values=['-'], low_memory=False)
df.columns = [c.lower() for c in df.columns]
df = df.replace('\*','', regex=True)

In [267]:
df ['difn-bfd3'] = pd.to_numeric(df['difn/tet- (cds only)']) - pd.to_numeric(df['bfd3/tet- (cds only)']) 
df ['difn-bfd6'] = pd.to_numeric(df['difn/tet- (cds only)']) - pd.to_numeric(df['bfd6/tet- (cds only)']) 
df ['difn-pf'] = pd.to_numeric(df['difn/tet- (cds only)']) - pd.to_numeric(df['pf/tet- (cds only)']) 

In [268]:
#add rnaseq results
#mono_df = pd.read_csv('/Users/goldriev/keep/store/PHD/Monomorph_wet/Clonal_selection/BHI_selection/RNA_seq/salmon/Results/', sep=",", na_values=['-'], low_memory=False)
#mono_df.columns = [c.lower() for c in mono_df.columns]
#mono_df = mono_df.replace('\*','', regex=True)
#mono_df = mono_df.fillna(0)
#mono_df["gene id"] = mono_df["rn"]
#df = pd.merge(df, mono_df, how="left", on=["gene id"])

In [269]:
#Add list of sif genes
sifs = {'gene id':("Tb927.2.1810", "Tb927.2.2720", "Tb927.2.4020", "Tb927.3.4560", 
        "Tb927.4.670", "Tb927.4.3620", "Tb927.4.3630", "Tb927.4.3640", 
        "Tb927.4.3650", "Tb927.5.3580", "Tb927.6.2300", "Tb927.6.2360", 
        "Tb927.7.2100", "Tb927.7.7160", "Tb927.8.2860", "Tb927.9.4080", 
        "Tb927.9.7550", "Tb927.9.13530", "Tb927.10.5930", "Tb927.10.5940", 
        "Tb927.10.5950", "Tb927.10.12100", "Tb927.10.15020", "Tb927.10.16120", 
        "Tb927.11.290", "Tb927.11.300", "Tb927.11.750", "Tb927.11.760", 
        "Tb927.11.1640", "Tb927.11.2250", "Tb927.11.3650", "Tb927.11.4610", 
        "Tb927.11.6600", "Tb927.11.11470", "Tb927.11.11480", "Tb927.10.12090", 
        "Tb927.1.1930", "Tb927.11.9270", "Tb927.6.4220", "Tb927.8.1530", "Tb927.10.1740", 
        "Tb927.10.1750", "Tb927.10.2030", "Tb927.10.2530", "Tb927.10.12110", "Tb927.11.1480", 
        "Tb927.11.6610", "Tb927.7.2660", "Tb927.4.5390", "Tb927.8.6930", "Tb927.9.6100", 
        "Tb927.9.6090", "Tb927.8.7020", "Tb927.11.2500", "Tb927.11.12850", "Tb927.8.2780")}
sifs = pd.DataFrame(sifs)
sifs['QS_Pathway'] = "Y"
df = pd.merge(df, sifs, how="left", on=["gene id"])

In [270]:
#Add target

#Add list of sif genes
target = {'gene id':("Tb927.8.1530", "Tb927.5.2580", "Tb927.2.4020", "Tb927.11.6600", "Tb927.4.3650", "Tb927.11.3400")}
target = pd.DataFrame(target)
target['target'] = "Y"
df = pd.merge(df, target, how="left", on=["gene id"])

In [271]:
#Add dnds results
directory = "~/Desktop/"
code = pd.read_table(directory + "codeml.txt", sep = "_", names = ["source_id", "txt", "dn", "dn/ds"])
dnds = code['dn'].str.split(' ',expand=True)[[5,6,7,8,9,10]]
dnds["source_id"] = code["source_id"]
dnds = dnds.rename({5:'background', 6:'ev_a', 7:'ev_b', 8:'ivmt1', 9:'ovi', 10:'botat'}, axis='columns')
dnds[["background", "ev_a", "ev_b", "ivmt1", "ovi", "botat"]] = dnds[["background", "ev_a", "ev_b", "ivmt1", "ovi", "botat"]].apply(pd.to_numeric, errors='coerce')
dnds[['gene id', 'type']] = dnds['source_id'].str.split(':', n=1, expand=True)
df = pd.merge(df, dnds, how="left", on=["gene id"])

In [272]:
#Add snpeff results
wd = "/Volumes/matthews/Guy/Raw_data/monomorph/data/snpeff/lineage_specific"
eva = pd.read_table(wd + '/T.b.evansitypeA.vcf.dir/snpEff_genes.txt', sep=" ", na_values=['-'], low_memory=False, header = None).iloc[1:][0].str.split('\t', expand=True)
eva.columns = eva.iloc[0]
eva = eva[1:].add_suffix('_eva')
eva[['gene id', 'type']] = eva['TranscriptId_eva'].str.split(':', n=1, expand=True)
eva.drop('type', axis=1, inplace=True)
evb = pd.read_table(wd + '/T.b.evansitypeB.vcf.dir/snpEff_genes.txt', sep=" ", na_values=['-'], low_memory=False, header = None).iloc[1:][0].str.split('\t', expand=True)
evb.columns = evb.iloc[0]
evb = evb[1:].add_suffix('_evb')
evb[['gene id', 'type']] = evb['TranscriptId_evb'].str.split(':', n=1, expand=True)
evb.drop('type', axis=1, inplace=True)
evc = pd.read_table(wd + '/T.b.evansitypeC.vcf.dir/snpEff_genes.txt', sep=" ", na_values=['-'], low_memory=False, header = None).iloc[1:][0].str.split('\t', expand=True)
evc.columns = evc.iloc[0]
evc = evc[1:].add_suffix('_evc')
evc[['gene id', 'type']] = evc['TranscriptId_evc'].str.split(':', n=1, expand=True)
evc.drop('type', axis=1, inplace=True)
ovi = pd.read_table(wd + '/T.b.equitypeOVI.vcf.dir/snpEff_genes.txt', sep=" ", na_values=['-'], low_memory=False, header = None).iloc[1:][0].str.split('\t', expand=True)
ovi.columns = ovi.iloc[0]
ovi = ovi[1:].add_suffix('_ovi')
ovi[['gene id', 'type']] = ovi['TranscriptId_ovi'].str.split(':', n=1, expand=True)
ovi.drop('type', axis=1, inplace=True)
botat = pd.read_table(wd + '/T.b.equitypeBOTAT.vcf.dir/snpEff_genes.txt', sep=" ", na_values=['-'], low_memory=False, header = None).iloc[1:][0].str.split('\t', expand=True)
botat.columns = botat.iloc[0]
botat = botat[1:].add_suffix('_botat')
botat[['gene id', 'type']] = botat['TranscriptId_botat'].str.split(':', n=1, expand=True)
botat.drop('type', axis=1, inplace=True)
pleo = pd.read_table('/Volumes/matthews/goldriev/monomorph/snpEff_genes.txt', sep=" ", na_values=['-'], low_memory=False, header = None).iloc[1:][0].str.split('\t', expand=True)
pleo.columns = pleo.iloc[0]
pleo = pleo[1:].add_suffix('_pleo')
pleo[['gene id', 'type']] = pleo['TranscriptId_pleo'].str.split(':', n=1, expand=True)
pleo.drop('type', axis=1, inplace=True)

dfList = [df, eva, evb, evc, ovi, botat, pleo]
df = reduce(lambda left,right: left.merge(right, how="left", on='gene id'), dfList)

In [273]:
df[['background','ev_a', 'ev_b', 'ivmt1', 'ovi', 'botat']] = df[['background','ev_a', 'ev_b', 'ivmt1', 'ovi', 'botat']].apply(pd.to_numeric)
filtered = df[(df['is pseudo'] != "Yes")]

In [274]:
filtered = filtered[~filtered['product description'].str.contains('variant surface')]

In [275]:
filtered = filtered[filtered['chromosome'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10','11'])] 

In [276]:
filtered[['ev_a', 'variants_impact_HIGH_pleo', 'variants_impact_MODERATE_eva', 'variants_impact_MODERATE_evb', 'variants_impact_MODERATE_evc', 'variants_impact_MODERATE_ovi', 'variants_impact_MODERATE_botat']] = filtered[['ev_a', 'variants_impact_HIGH_pleo', 'variants_impact_MODERATE_eva', 'variants_impact_MODERATE_evb', 'variants_impact_MODERATE_evc', 'variants_impact_MODERATE_ovi', 'variants_impact_MODERATE_botat']].apply(pd.to_numeric)
dnds_filtered = filtered[(filtered['background'] < 1) & (filtered['variants_impact_HIGH_pleo'] == 0 )]
dnds_filtered[['variants_impact_HIGH_eva', 'variants_impact_HIGH_evb', 'variants_impact_HIGH_evc', 'variants_impact_HIGH_ovi', 'variants_impact_HIGH_botat']] = dnds_filtered[['variants_impact_HIGH_eva', 'variants_impact_HIGH_evb', 'variants_impact_HIGH_evc', 'variants_impact_HIGH_ovi', 'variants_impact_HIGH_botat']].apply(pd.to_numeric)

/var/folders/k5/mm9d16t101v50p5b4qwh8hfh02tcwb/T/ipykernel_35800/2081220531.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dnds_filtered[['variants_impact_HIGH_eva', 'variants_impact_HIGH_evb', 'variants_impact_HIGH_evc', 'variants_impact_HIGH_ovi', 'variants_impact_HIGH_botat']] = dnds_filtered[['variants_impact_HIGH_eva', 'variants_impact_HIGH_evb', 'variants_impact_HIGH_evc', 'variants_impact_HIGH_ovi', 'variants_impact_HIGH_botat']].apply(pd.to_numeric)


In [277]:
dnds_filtered = dnds_filtered[(dnds_filtered['ev_a'] > 1) | (dnds_filtered['ev_b'] > 1) | (dnds_filtered['ivmt1'] > 1) | (dnds_filtered['ovi'] > 1) | (dnds_filtered['botat'] > 1)]
dnds_filtered = dnds_filtered[(dnds_filtered['variants_impact_MODERATE_eva'] > 0) | (dnds_filtered['variants_impact_MODERATE_evb'] > 0) | (dnds_filtered['variants_impact_MODERATE_evc'] > 0) | (dnds_filtered['variants_impact_MODERATE_ovi'] > 0) | (dnds_filtered['variants_impact_MODERATE_botat'] > 0) | (dnds_filtered['variants_impact_HIGH_eva'] > 0) | (dnds_filtered['variants_impact_HIGH_evb'] > 0) | (dnds_filtered['variants_impact_HIGH_evc'] > 0) | (dnds_filtered['variants_impact_HIGH_ovi'] > 0) | (dnds_filtered['variants_impact_HIGH_botat'] > 0)]

In [278]:
dnds_filtered = dnds_filtered[(dnds_filtered['difn-bfd3'] <= 0) & (dnds_filtered['difn-bfd6'] <= 0) & (dnds_filtered['difn-pf'] <= 0)]
dnds_filtered[['difn/tet- (cds only)']] = dnds_filtered[['difn/tet- (cds only)']].apply(pd.to_numeric)
dnds_filtered = dnds_filtered[(dnds_filtered['difn/tet- (cds only)'] < -1.5)]

,gene id,# exons in gene,# exons in transcript,# transcripts,annotated 3' utr length,annotated 5' utr length,gene strand,gene type,is pseudo,transcript length,...,variants_effect_splice_acceptor_variant_pleo,variants_effect_splice_donor_variant_pleo,variants_effect_splice_region_variant_pleo,variants_effect_start_lost_pleo,variants_effect_start_retained_variant_pleo,variants_effect_stop_gained_pleo,variants_effect_stop_lost_pleo,variants_effect_stop_retained_variant_pleo,variants_effect_synonymous_variant_pleo,variants_effect_upstream_gene_variant_pleo
1859,Tb927.1.1060,1,1,1,832.0,67.0,forward,protein coding gene,No,1817,...,0,0,0,0,0,0,0,0,1,4
1880,Tb927.1.1330,1,1,1,NaN,19.0,forward,protein coding gene,No,1240,...,0,0,0,0,0,0,0,0,0,21
1910,Tb927.1.1750,1,1,1,16.0,44.0,forward,protein coding gene,No,1473,...,0,0,0,0,0,0,0,0,6,25
1915,Tb927.1.1820,1,1,1,42.0,152.0,forward,protein coding gene,No,1340,...,0,0,0,0,0,0,0,0,5,129
1964,Tb927.1.2570,1,1,1,NaN,23.0,forward,protein coding gene,No,2972,...,0,0,0,0,0,0,0,0,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11479,Tb927.9.8450,1,1,1,275.0,92.0,reverse,protein coding gene,No,970,...,0,0,0,0,0,0,0,0,3,49
11495,Tb927.9.8680,1,1,1,NaN,51.0,reverse,protein coding gene,No,867,...,0,0,0,0,0,0,0,0,3,79
11555,Tb927.9.9550,1,1,1,388.0,112.0,reverse,protein coding gene,No,1139,...,0,0,0,0,0,0,0,0,3,62
11563,Tb927.9.9660,1,1,1,926.0,81.0,reverse,protein coding gene,No,2063,...,0,0,0,0,0,0,0,0,3,29


In [264]:
dnds_filtered.to_csv('~/Desktop/dnds_targets.csv', index = False)

In [290]:
qs_filtered = filtered
qs_filtered[['ev_a', 'variants_impact_HIGH_pleo', 'variants_impact_MODERATE_eva', 'variants_impact_MODERATE_evb', 'variants_impact_MODERATE_evc', 'variants_impact_MODERATE_ovi', 'variants_impact_MODERATE_botat']] = qs_filtered[['ev_a', 'variants_impact_HIGH_pleo', 'variants_impact_MODERATE_eva', 'variants_impact_MODERATE_evb', 'variants_impact_MODERATE_evc', 'variants_impact_MODERATE_ovi', 'variants_impact_MODERATE_botat']].apply(pd.to_numeric)
qs_filtered[['variants_impact_HIGH_eva', 'variants_impact_HIGH_evb', 'variants_impact_HIGH_evc', 'variants_impact_HIGH_ovi', 'variants_impact_HIGH_botat']] = qs_filtered[['variants_impact_HIGH_eva', 'variants_impact_HIGH_evb', 'variants_impact_HIGH_evc', 'variants_impact_HIGH_ovi', 'variants_impact_HIGH_botat']].apply(pd.to_numeric)
qs_filtered = qs_filtered[(qs_filtered['variants_impact_HIGH_pleo'] == 0 )]
qs_filtered = filtered[(filtered['variants_impact_MODERATE_eva'] > 0) | (filtered['variants_impact_MODERATE_evb'] > 0) | (filtered['variants_impact_MODERATE_evc'] > 0) | (filtered['variants_impact_MODERATE_ovi'] > 0) | (filtered['variants_impact_MODERATE_botat'] > 0) | (filtered['variants_impact_HIGH_eva'] > 0) | (dnds_filtered['variants_impact_HIGH_evb'] > 0) | (dnds_filtered['variants_impact_HIGH_evc'] > 0) | (dnds_filtered['variants_impact_HIGH_ovi'] > 0) | (filtered['variants_impact_HIGH_botat'] > 0)]
qs_filtered = qs_filtered[(qs_filtered['QS_Pathway'] == "Y")]
qs_filtered.to_csv('~/Desktop/qs_targets.csv', index = False)